In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs4
import requests
import time
import pandas as pd
from random import randint
import pymongo

In [2]:
# will be looking at top 1000 movies for each of the 3 years: 2017, 2018, 2019
# imdb displays 50 movies per page to get 1000 movies we need to make 20 calls for each year (60 calls in total)
#url_2017 = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=1&ref_=adv_nxt'
url_2018 = 'https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&start=1&ref_=adv_nxt'
#url_2019 = 'https://www.imdb.com/search/title/?release_date=2019-01-01,2019-12-31&start=1&ref_=adv_nxt'

In [3]:
# Retrieve page with the requests module
response = requests.get(url_2018)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs4(response.text, 'html.parser')

In [5]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Released between 2018-01-01 and 2018-12-31
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
  

In [6]:
movies = soup.find_all('div', class_='lister-item mode-advanced')
#len(movies)
movies[0]

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt6548228"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt6548228/"> <img alt="Castle Rock" class="loadlate" data-tconst="tt6548228" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYWQ0ZmE0ZjEtZDZjOC00ODQ4LTk1MGQtNTk3ZDdlY2RiODg1XkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt6548228/">Castle Rock</a>
<span class="lister-item-year text-muted unbold">(2018– )</span>
</h3>
<p class="text-muted">
<span class="certificate">14+</span>
<span class="ghost">|</span>
<span class="runtime">60 min</span>
<span class="ghost">|</span>
<span cla

In [30]:
##################### SAMPLING 50 RECORDS ##################################3

def process_imdb_one_page(movies, year):

    for m in movies:

        movie_name = m.h3.a.text
        movie_year = year #m.h3.find('span', class_="lister-item-year").text
        try:
            movie_rating_imdb = float(m.strong.text)
        except:
            movie_rating_imdb = -1
        try:
            movie_duration = m.p.find('span', class_="runtime").text
        except:
            movie_duration = -1
        try:
            movie_genre = m.p.find('span', class_="genre").text
        except:
            movie_genre = 'N/A'
        try:
            movie_description = m.find_all('p')[1].text
        except:
            movie_description = 'N/A'
        try:
            movie_score_meta = int(m.find('span', class_ = 'metascore').text)
        except:
            movie_score_meta = -1 #if no meta score assign -1 value

        try:
            movie_votes = int(m.find('span', attrs = {'name':'nv'})['data-value'])
        except:
            movie_votes = -1

        movie_list_info.append({"move_name":movie_name,
                               "movie_year":movie_year,
                               "movie_rating_imdb":movie_rating_imdb,
                               "movie_duration":movie_duration,
                               "movie_genre":movie_genre,
                               "movie_description":movie_description,
                               "movie_score_meta":movie_score_meta,
                               "movie_votes":movie_votes})

    
    #print(f'{movie_name}, {movie_genre}, {movie_duration}, {movie_year}, {movie_rating_imdb}, {movie_score_meta}, {movie_votes}')
    #print(movie_description)

In [31]:
### call one page function and save as data frame, will use in the loop later
#movie_list_info = []
#process_imdb_one_page(movies, 2018)
#movies_df = pd.DataFrame(movie_list_info)

In [32]:
movies_df.head(10)

,move_name,movie_description,movie_duration,movie_genre,movie_rating_imdb,movie_score_meta,movie_votes,movie_year
0,The Marvelous Mrs. Maisel,\n A housewife in the 1960s decides to beco...,57 min,"\nComedy, Drama",8.8,-1,54050,2017
1,Jumanji: Welcome to the Jungle,\n Four teenagers are sucked into a magical...,119 min,"\nAction, Adventure, Comedy",6.9,58,266012,2017
2,Star Wars: Episode VIII - The Last Jedi,\n Rey develops her newly discovered abilit...,152 min,"\nAction, Adventure, Fantasy",7.1,85,490391,2017
3,La Casa de Papel,\n A group of unique robbers assault the Fa...,70 min,"\nAction, Crime, Mystery",8.5,-1,177832,2017
4,Riverdale,\n While navigating the troubled waters of ...,45 min,"\nCrime, Drama, Mystery",7.1,-1,98562,2017
5,The Good Doctor,"\n Shaun Murphy, a young surgeon with autis...",41 min,\nDrama,8.2,-1,50129,2017
6,Thor: Ragnarok,"\n Imprisoned on the planet Sakaar, Thor mu...",130 min,"\nAction, Adventure, Comedy",7.9,74,522608,2017
7,Mindhunter,"\n Set in the late 1970s, two FBI agents ar...",60 min,"\nCrime, Drama, Thriller",8.6,-1,167686,2017
8,The Handmaid's Tale,"\n Set in a dystopian future, a woman is fo...",60 min,"\nDrama, Sci-Fi, Thriller",8.5,-1,142581,2017
9,SEAL Team,\n The lives of the elite Navy SEALs as the...,43 min,"\nAction, Drama, War",7.4,-1,12538,2017


In [37]:
### prepare url strings that will be called 
years = ['2017', '2018', '2019']
pages = [str(i*50+1) for i in range(0,20)]
urls = []

for y in years:
    for p in pages:
        url = {"year":y, "url":f'https://www.imdb.com/search/title/?release_date={y}-01-01,{y}-12-31&start={p}&ref_=adv_nxt'}
        urls.append(url)
        
urls

[{'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=1&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=51&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=101&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=151&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=201&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=251&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=301&ref_=adv_nxt'},
 {'year': '2017',
  'url': 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=351&ref_=adv_nxt'},
 {'year': '

In [38]:
movie_list_info = []

for u in urls:
    
    url = u["url"]
    year = u["year"]
    sleep_time = randint(1,8)
    print (f'Processsing: {url} , {year}')
    
    response = requests.get(url)
    soup = bs4(response.text, 'html.parser')
    movies = soup.find_all('div', class_='lister-item mode-advanced')
    process_imdb_one_page(movies, year)
    
    print (f'.....Sleeping for {sleep_time}')
    time.sleep(sleep_time)
    
    

Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=1&ref_=adv_nxt , 2017
.....Sleeping for 2
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=51&ref_=adv_nxt , 2017
.....Sleeping for 5
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=101&ref_=adv_nxt , 2017
.....Sleeping for 6
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=151&ref_=adv_nxt , 2017
.....Sleeping for 6
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=201&ref_=adv_nxt , 2017
.....Sleeping for 8
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=251&ref_=adv_nxt , 2017
.....Sleeping for 2
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&start=301&ref_=adv_nxt , 2017
.....Sleeping for 2
Processsing: https://www.imdb.com/search/title/?release_date=2017-01-01,

In [39]:
movies_df = pd.DataFrame(movie_list_info)
movies_df.describe()

,movie_rating_imdb,movie_score_meta,movie_votes
count,3000.000000,3000.000000,3000.000000
mean,6.443433,22.141333,18654.223333
std,1.408187,30.591785,53493.906713
min,-1.000000,-1.000000,-1.000000
25%,5.700000,-1.000000,903.250000
50%,6.600000,-1.000000,3171.500000
75%,7.300000,51.000000,11836.000000
max,9.900000,96.000000,731468.000000


In [40]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.movies_db
collection = db.imdb_data

collection.insert_many(movie_list_info)
